#### **Análise de negócios**

#### **Observações**

#### **Conteúdo - Bases e Notebook da aula**

Github:  

https://github.com/FIAP/Pos_Tech_DTAT/tree/Analise-de-Negocios/Analise%20de%20Negocios  

Série Histórica de Preços de Combustíveis e de GLP:  

https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

#### **Importação de pacotes e bibliotecas**

In [ ]:
# Importar biblioteca completa
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import sys
import zipfile  
import io       
import time

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv

#### **Funções (def)**

In [ ]:
def encontrar_links(soup):
    
    """
    Encontra dinamicamente os links de download na seção 
    'Combustíveis automotivos'.
    """
    
    # Encontra o cabeçalho <h3> que contém o texto "Combustíveis automotivos"
    heading = soup.find(lambda tag: tag.name == 'h3' and 'Combustíveis automotivos' in tag.get_text())
    
    links_para_baixar = []
    
    if not heading:
        print("Erro: Não foi possível encontrar a seção 'Combustíveis automotivos' no HTML da página")
        return links_para_baixar

    # A lista <ul> com os links é o próximo "irmão" (sibling) da tag <h3>
    ul_tag = heading.find_next_sibling('ul')
    
    if not ul_tag:
        print("Erro: Não foi possível encontrar a lista <ul> após o cabeçalho")
        return links_para_baixar

    # Encontra todas as tags <a> (links) dentro desta lista <ul>
    a_tags = ul_tag.find_all('a')
    
    for a_tag in a_tags:
        url = a_tag.get('href')
        if url:
            links_para_baixar.append(url)
            
    return links_para_baixar

In [ ]:
def processar_arquivo(url, pasta_destino, max_retries, retry_delay):

    """
    Baixa um arquivo com retentativas. Se for .zip, extrai. Se .csv, salva.
    """

    nome_arquivo = os.path.basename(url)
    
    for attempt in range(max_retries):
        try:
            
            if nome_arquivo.endswith('.zip'):
                print(f"Baixando e extraindo {nome_arquivo} (Tentativa {attempt + 1}/{max_retries})...")
                
                response = requests.get(url) 
                response.raise_for_status()
                
                with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
                    zf.extractall(pasta_destino)
                    nomes_extraidos = zf.namelist()
                    print(f"-> Concluído. Extraído(s): {', '.join(nomes_extraidos)}\n")
                
                break 

            else:
                print(f"Baixando {nome_arquivo} (Tentativa {attempt + 1}/{max_retries})...")
                caminho_local = os.path.join(pasta_destino, nome_arquivo)
                
                with requests.get(url, stream=True) as r:
                    r.raise_for_status()
                    with open(caminho_local, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192): 
                            f.write(chunk)
                            
                print(f"-> Concluído. Salvo em: {caminho_local}\n")
                
                break 

        except requests.exceptions.RequestException as e:
            print(f"Erro na Tentativa {attempt + 1}: {e}")
            if attempt + 1 < max_retries:
                print(f"Aguardando {retry_delay}s antes de tentar novamente...")
                time.sleep(retry_delay)
            else:
                print(f"Falha ao baixar {nome_arquivo} após {max_retries} tentativas. Pulando\n")
        
        except zipfile.BadZipFile:
            print(f"Erro: O arquivo {nome_arquivo} não é um .zip válido ou está corrompido. Pulando\n")
            break 

        except Exception as e:
            print(f"Erro inesperado ao processar {nome_arquivo}: {e}. Pulando\n")
            break 

#### **Credenciais**

In [ ]:
load_dotenv()

# Credenciais do PostgreSQL
usuario_pg = os.getenv("POSTGRES_USER")
senha_pg = os.getenv("POSTGRES_PASSWORD")
host_pg = os.getenv("POSTGRES_HOST")
porta_pg = os.getenv("POSTGRES_PORT")
banco_pg = os.getenv("POSTGRES_DB")

#### **Aula 1 - Processos e formas de análise**

#### **Aula 2 - Ligação com bancos de dados**

In [ ]:

MAX_RETRIES = 3  # Número máximo de tentativas por arquivo
RETRY_DELAY = 5  # Segundos de espera entre as tentativas

# URL da página de onde vamos extrair os links
PAGE_URL = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis'

# Pasta para onde os arquivos serão baixados
DOWNLOAD_DIR = 'arquivos_combustiveis_automotivos'

# 1. Criar a pasta de download se ela não existir
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# 2. Baixar o HTML da página da ANP
print(f"Acessando a página: {PAGE_URL}")
try:
    response = requests.get(PAGE_URL)
    # Verifica se a requisição foi bem-sucedida (código 200)
    response.raise_for_status()
    conteudo_html = response.text
    
except requests.exceptions.RequestException as e:
    print(f"Erro fatal ao acessar a página da ANP: {e}")
    print("Verifique sua conexão com a internet ou se a URL da ANP mudou")
    sys.exit(1) # Termina o script se não conseguir acessar a página

# 3. Analisar (parse) o HTML
soup = BeautifulSoup(conteudo_html, 'html.parser')

# 4. Encontrar os links
links = encontrar_links(soup)

if not links:
    print("Nenhum link encontrado para baixar")

else:
    print(f"Encontrados {len(links)} arquivos para baixar na seção 'Combustíveis automotivos'")
    print()

    # 5. Processar (baixar ou extrair) cada arquivo
    for url in links:
        processar_arquivo(url, DOWNLOAD_DIR,MAX_RETRIES,RETRY_DELAY) 
        
    print()
    print("Download e processamento de todos os arquivos concluído")
    print(f"Os arquivos estão salvos em: {os.path.abspath(DOWNLOAD_DIR)}")